# Steady-State Distillation  Model

In [1]:
import numpy as np
from scipy.optimize import fsolve

In [2]:
N_COMPS = 5

In [3]:
class Stream:
    
    n_streams = 0
    
    def __init__(self, n_comps=1, name=None):
        # the stream member x_var will be as follows:
        # [0] flow rate
        # [1] temperature
        # [2:-1] fraction of each component. if flow is in mass units, these are mass fractions
        self.n_comps = n_comps
        self.n_vars = 2 + n_comps
        self.stream_num = Stream.n_streams
        Stream.n_streams += 1
        self.xvar = None
        if name is None:
            self.name = 'Stream' + str(self.stream_num)
        else:
            self.name = name
        
    def __str__(self):
        s = 'Stream name: {}\n'.format(self.name)
        s = s + 'Stream number: {}\n'.format(self.stream_num)
        if self.xvar is None:
            s = s + 'Variables not set\n'
        else:
            s = s + 'Flow rate: {}\n'.format(self.xvar[0])
            s = s + 'Temperature: {}\n'.format(self.xvar[1])
            s = s + 'Fractions: {}\n'.format(self.xvar[2:])
        return s

In [4]:
class Equipment():
    n_equipments = 0

In [28]:
class Mixer(Equipment):
    
    n_mixers = 0
    
    def __init__(self, streams_in, streams_out, name=None):
        self.streams_in = streams_in
        self.streams_out = streams_out
        self.n_in = len(self.streams_in)
        self.n_out = len(self.streams_out)
        self.n_eqns = (self.streams_in[0].n_comps + 1) * self.n_out + 1
        self.mixer_num = Mixer.n_mixers
        Mixer.n_mixers += 1
        self.equipment_num = Equipment.n_equipments
        Equipment.n_equipments += 1
        self.eqns = None
        if name is None:
            self.name = 'Mixer' + str(self.mixer_num)
        else:
            self.name = name

    def __str__(self):
        s = 'Equipment name: {}\n'.format(self.name)
        s = s + 'Equipment number: {}\n'.format(self.equipment_num)
        s = s + 'Mixer number: {}\n'.format(self.mixer_num)
        s_in = list()
        for stream in self.streams_in:
            s_in.append(stream.name)
        s = s + 'Streams in: {}\n'.format(','.join(s_in))
        s_out = list()
        for stream in self.streams_out:
            s_out.append(stream.name)
        s = s + 'Streams out: {}\n'.format(', '.join(s_out))
        s = s + 'Number of equations: {}\n'.format(self.n_eqns)
        if self.eqns is None:
            s = s + 'Equations not set\n'
        else:
            s = s + 'Equations: {}\n'.format(self.eqns[:])
        return s
    
    def calculate(self):
        # total mass balance: 1 equation
        total_in = 0
        for s in self.streams_in:
            total_in += s.xvar[0]
        total_out = 0
        for s in self.streams_out:
            total_out += s.xvar[0]
        self.eqns[0] = total_in - total_out
        # heat balance (simple: temp out is weighted average of temp in. n_out equations)
        t_avg_in = 0
        for s in self.streams_in:
            t_avg_in += s.xvar[0] * s.xvar[1]
        t_avg_in = t_avg_in / total_in
        eq_n = 0
        for s in self.streams_out:
            eq_n += 1
            self.eqns[eq_n] = t_avg_in - s.xvar[1]
        # component balances (n_out * n_comps equations)
        for i_comp in range(self.streams_in[0].n_comps):
            comp_in = 0
            for s in self.streams_in:
                comp_in += s.xvar[2 + i_comp] * s.xvar[0]
            comp_in = comp_in / total_in
            for s in self.streams_out:
                eq_n += 1
                self.eqns[eq_n] = comp_in - s.xvar[2 + i_comp]
        return

In [29]:
class Specify():
    
    n_specs = 0
    
    def __init__(self, name=None, flow=False, temperature=False, fraction=False, 
                 stream=None, comp_num=None, value=None):
        self.spec_num = Specify.n_specs
        Specify.n_specs += 1
        if name is None:
            self.name = 'Specification' + str(self.spec_num)
        else:
            self.name = name
        assert flow + temperature + fraction == 1, 'Too many/few specifications for ' + self.name
            
        self.flow = flow
        self.temperature = temperature
        self.fraction = fraction
        self.stream = stream
        self.comp_num = comp_num
        self.value = value
        self.n_eqns = 1
        self.eqns = None

    def __str__(self):
        s = 'Specification name: {}\n'.format(self.name)
        s = s + 'Stream : {}\n'.format(self.stream.name)
        s = s + 'Number of equations: {}\n'.format(self.n_eqns)
        if self.flow:
            s = s + 'Flow: {}\n'.format(self.value)
        elif self.temperature:
            s = s + 'Temperature: {}\n'.format(self.value)
        elif self.fraction:
            s = s + 'Fraction: component {}, value: {}\n'.format(self.comp_num, self.value)            
            
        if self.eqns is None:
            s = s + 'Equations not set\n'
        else:
            s = s + 'Equations: {}\n'.format(self.eqns[:])
        return s
    
    def calculate(self):
        if self.flow:
            self.eqns[0] = self.value -  self.stream.xvar[0]
        if self.temperature:
            self.eqns[0] = self.value -  self.stream.xvar[1]
        if self.fraction:
            self.eqns[0] = self.value -  self.stream.xvar[2 + self.comp_num]
        return

In [30]:
condensate = Stream(n_comps=N_COMPS, name='Condensate')
reflux = Stream(n_comps=N_COMPS, name='Reflux')
top_product = Stream(n_comps=N_COMPS, name='Top product')

In [31]:
stream_list = [condensate, reflux, top_product]

In [32]:
n_vars = 0
for stream in stream_list:
    n_vars += stream.n_vars

In [33]:
print('number of variables: {}'.format(n_vars))

number of variables: 21


In [34]:
condensate_splitter = Mixer(streams_in=[condensate], streams_out=[reflux, top_product], name='Condensate Splitter')

In [35]:
equipment_list = [condensate_splitter]

In [36]:
xvar = np.ones(n_vars, dtype=np.float64)

In [37]:
xvar

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1.])

In [38]:
def map_x_to_stream(x):
    idx = 0
    for stream in stream_list:
        stream.xvar = x[idx:idx+stream.n_vars]
        idx += stream.n_vars

In [39]:
map_x_to_stream(xvar)

In [40]:
xvar

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1.])

NOTE: The above `xvar` will be passed to `fsolve` as an initial guess. `fsolve` will pass some other internal variable to the equation calculating function, `process_eqns`. Thus, `process_eqns` will have to re-map that variable to the `Stream` members.

In [41]:
xvar

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1.])

In [42]:
condensate_flow_spec = Specify(flow=True, stream=condensate, value=10)
condensate_temp_spec = Specify(temperature=True, stream=condensate, value=100)
condensate_x1_spec = Specify(fraction=True, stream=condensate, comp_num=0,value=0.2)
condensate_x2_spec = Specify(fraction=True, stream=condensate, comp_num=1,value=0.1)
condensate_x3_spec = Specify(fraction=True, stream=condensate, comp_num=2,value=0.2)
condensate_x4_spec = Specify(fraction=True, stream=condensate, comp_num=3,value=0.1)
condensate_x5_spec = Specify(fraction=True, stream=condensate, comp_num=4,value=0.4)
reflux_flow_spec = Specify(flow=True, stream=reflux, value=7)

In [43]:
spec_list = [condensate_flow_spec, condensate_temp_spec, condensate_x1_spec, condensate_x2_spec, 
             condensate_x3_spec, condensate_x4_spec, condensate_x5_spec, 
             reflux_flow_spec]

In [44]:
equation_list = equipment_list + spec_list

In [45]:
n_eqns = 0
for equation in equation_list:
    n_eqns += equation.n_eqns

In [46]:
print('number of equations: {}'.format(n_eqns))

number of equations: 21


In [47]:
eqns = np.ones(n_vars, dtype=np.float64)

In [48]:
eqns

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1.])

In [49]:
idx = 0
for equation in equation_list:
    equation.eqns = eqns[idx:idx+equation.n_eqns]
    idx += equation.n_eqns

In [50]:
for equation in equation_list:
    equation.calculate()

In [51]:
eqns

array([-1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  9. , 99. , -0.8, -0.9, -0.8, -0.9, -0.6,  6. ])

This is just a test of the equation calculations.

In [52]:
print(xvar)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [53]:
def process_eqns(xvar):
    # map xvar to the class Stream members. this is being done at each call
    # in case fsolve passes a different variable (memory locations) from time to time.
    # investigate and do this mapping only during the first call to this function
    # if that works.
    map_x_to_stream(xvar)
    
    for equation in equation_list:
        equation.calculate()

    # fsolve apparently expects a list to be returned, not a numpy array, though it is not documented.
    return eqns.tolist()

In [54]:
xvar

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1.])

In [55]:
x_solution = fsolve(process_eqns, xvar)

In [56]:
x_solution

array([ 10. , 100. ,   0.2,   0.1,   0.2,   0.1,   0.4,   7. , 100. ,
         0.2,   0.1,   0.2,   0.1,   0.4,   3. , 100. ,   0.2,   0.1,
         0.2,   0.1,   0.4])

In [57]:
map_x_to_stream(x_solution)

In [58]:
eqns

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])

In [60]:
print(condensate)

Stream name: Condensate
Stream number: 3
Flow rate: 10.0
Temperature: 100.0
Fractions: [0.2 0.1 0.2 0.1 0.4]



In [61]:
print(reflux)

Stream name: Reflux
Stream number: 4
Flow rate: 7.0
Temperature: 100.0
Fractions: [0.2 0.1 0.2 0.1 0.4]



In [62]:
print(top_product)

Stream name: Top product
Stream number: 5
Flow rate: 2.999999999999999
Temperature: 100.0
Fractions: [0.2 0.1 0.2 0.1 0.4]

